In [114]:
import numpy as np 
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/book-recommendation-dataset/Ratings.csv
/kaggle/input/book-recommendation-dataset/Users.csv
/kaggle/input/book-recommendation-dataset/classicRec.png
/kaggle/input/book-recommendation-dataset/Books.csv
/kaggle/input/book-recommendation-dataset/DeepRec.png
/kaggle/input/book-recommendation-dataset/recsys_taxonomy2.png


In [115]:
books = pd.read_csv("/kaggle/input/book-recommendation-dataset/Books.csv")
users = pd.read_csv("/kaggle/input/book-recommendation-dataset/Users.csv")
ratings = pd.read_csv("/kaggle/input/book-recommendation-dataset/Ratings.csv")

/tmp/ipykernel_33/3593383676.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("/kaggle/input/book-recommendation-dataset/Books.csv")


In [116]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [117]:
books.duplicated().sum(), ratings.duplicated().sum(), users.duplicated().sum()

(0, 0, 0)

In [118]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [119]:
ratings_with_name = ratings.merge(books, on='ISBN')

# Popularity Based

In [120]:
num_ratings_data = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_ratings_data.rename(columns={'Book-Title': 'book_title', 'Book-Rating': 'num_rating'}, inplace=True)
num_ratings_data.head()

,book_title,num_rating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [121]:
avg_ratings_data = ratings_with_name.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avg_ratings_data.rename(columns={'Book-Title': 'book_title', 'Book-Rating': 'avg_rating'}, inplace=True)
avg_ratings_data.head()

,book_title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.25
1,Always Have Popsicles,0.00
2,Apple Magic (The Collector's series),0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00
4,Beyond IBM: Leadership Marketing and Finance ...,0.00


In [122]:
num_of_times_rated = 100
most_rated_books = num_ratings_data[num_ratings_data['num_rating'] >= num_of_times_rated]

In [123]:
most_rated_books.shape

(914, 2)

In [124]:
popular_books_data = most_rated_books.merge(avg_ratings_data, on='book_title')

In [125]:
top_n_books = 10
popular_books_data.sort_values('avg_rating', ascending=False).head(top_n_books)

,book_title,num_rating,avg_rating
283,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804
281,Harry Potter and the Goblet of Fire (Book 4),387,5.824289
726,The Little Prince,141,5.815603
284,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410
282,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441
218,Ender's Game (Ender Wiggins Saga (Paperback)),249,5.409639
280,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453
752,The Perks of Being a Wallflower,104,5.144231
697,The Hobbit : The Enchanting Prelude to The Lor...,281,5.007117
670,The Fellowship of the Ring (The Lord of the Ri...,368,4.948370


# Collaborative Based Filtering System

In [126]:
n_times_ratings_given = 200
users_given_ratings_more_than_n = ratings_with_name.groupby('User-ID')['Book-Rating'].count() > n_times_ratings_given

In [127]:
indexes_of_users = users_given_ratings_more_than_n[users_given_ratings_more_than_n].index

In [128]:
filtered_users = ratings_with_name[ratings_with_name['User-ID'].isin(indexes_of_users)]

In [129]:
n_of_times_book_recommended = 50
filtered_books = filtered_users.groupby('Book-Title')['Book-Rating'].count() >= n_of_times_book_recommended
filtered_books = filtered_books[filtered_books].index

In [130]:
final_books = filtered_users[filtered_users['Book-Title'].isin(filtered_books)]

In [131]:
books_per_user_ratings =  final_books.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating').fillna(0)

In [132]:
books_per_user_ratings

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [139]:
from sklearn.metrics.pairwise import cosine_similarity

In [140]:
similarity_scores = cosine_similarity(books_per_user_ratings)

In [135]:
similarity_scores.shape, books_per_user_ratings.shape

((706, 706), (706, 810))

# Recomendations

In [136]:
book_name = "Year of Wonders"

index = np.where(books_per_user_ratings.index == book_name)[0][0]
similar_books = sorted(list(enumerate(similarity_scores[index])), key=lambda x: x[1], reverse=True)[:10]
for book in similar_books:
    _book = books[books['Book-Title'] == books_per_user_ratings.index[book[0]]].drop_duplicates('Book-Title')
    print("Book Name: "+_book['Book-Title'].values[0],"\nBook Author: "+ _book['Book-Author'].values[0], "\nBook Publisher: "+_book['Publisher'].values[0])
    print()

Book Name: Year of Wonders 
Book Author: Geraldine Brooks 
Book Publisher: Penguin Books

Book Name: The Eight 
Book Author: Katherine Neville 
Book Publisher: Ballantine Books

Book Name: A Heartbreaking Work of Staggering Genius 
Book Author: Dave Eggers 
Book Publisher: Vintage Books USA

Book Name: Bridget Jones: The Edge of Reason 
Book Author: Helen Fielding 
Book Publisher: Penguin Books

Book Name: Drowning Ruth (Oprah's Book Club) 
Book Author: CHRISTINA SCHWARZ 
Book Publisher: Ballantine Books

Book Name: Midwives: A Novel 
Book Author: Chris Bohjalian 
Book Publisher: Vintage

Book Name: The Mists of Avalon 
Book Author: MARION ZIMMER BRADLEY 
Book Publisher: Del Rey

Book Name: The Sweet Potato Queens' Book of Love 
Book Author: JILL CONNER BROWNE 
Book Publisher: Three Rivers Press

Book Name: The Pilot's Wife : A Novel 
Book Author: Anita Shreve 
Book Publisher: Back Bay Books

Book Name: Into the Wild 
Book Author: Jon Krakauer 
Book Publisher: Anchor



In [138]:
# import pickle

# pickle.dump(pt,open('pt.pkl','wb'))
# pickle.dump(books,open('books.pkl','wb'))
# pickle.dump(similarity_scores,open('similarity_scores.pkl','wb'))